# This didn't work out that well (See topics)

In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from scipy.sparse import hstack
from scipy.special import logit, expit

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
import string

import gensim

import pickle

# Initiate Class Names and Open Files

In [2]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('/Users/randy/Documents/GitHub/Twitch_Chat_Harassment/toxic_comment_classification/jigsaw-toxic-comment-classification-challenge/cleaned_train.csv').fillna(' ')
test = pickle.load(open('/Users/randy/Documents/GitHub/Twitch_Chat_Harassment/chat_logs/callofduty_chat.p','rb'))

# Train Feature Engineering

In [3]:
#remove pucnt
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

train['comment_text'] = train['comment_text'].apply(lambda x: remove_punct(x))
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i am ...,0,0,0,0,0,0
2,000113f07ec002fd,hey man i am really not trying to edit war it ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i cannot make any real suggestions on imp...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0


In [4]:
#remove non-ascii characters
def remove_non_ascii(text):
    text  = "".join([char for char in text if char in string.printable])
    text = re.sub('[0-9]+', '', text)
    return text

train['comment_text'] = train['comment_text'].apply(lambda x: remove_non_ascii(x))
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww he matches this background colour i am ...,0,0,0,0,0,0
2,000113f07ec002fd,hey man i am really not trying to edit war it ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i cannot make any real suggestions on imp...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0


# Test Engineering

In [5]:
ar = np.array(range(0,len(test)))

In [6]:
test['id'] = ar

In [7]:
list_sentences_train = train['comment_text']
list_sentences_test = test['message']

In [8]:
all_text = pd.concat([list_sentences_train, list_sentences_test])

# Apply TF-IDF Vectorizer

In [25]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=50000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(list_sentences_train)
test_word_features = word_vectorizer.transform(list_sentences_test)

In [26]:
train_word_features


<159571x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 6828358 stored elements in Compressed Sparse Row format>

In [27]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
n = 100

lsa_train = TruncatedSVD(n)
train_doc_topic = lsa_train.fit_transform(train_word_features)

In [28]:
lsa_train.explained_variance_ratio_

array([0.00812863, 0.00682358, 0.0048498 , 0.00432348, 0.00345661,
       0.00330438, 0.00298955, 0.00278864, 0.0026418 , 0.00238343,
       0.00232347, 0.00223937, 0.00219528, 0.00211606, 0.00207611,
       0.0019727 , 0.00194736, 0.00190269, 0.00185285, 0.00181289,
       0.00176704, 0.00175484, 0.00168495, 0.00167156, 0.00165225,
       0.00163949, 0.00162347, 0.00160811, 0.00154302, 0.00152122,
       0.00150391, 0.00148751, 0.00146892, 0.00145291, 0.00144703,
       0.00142692, 0.00140934, 0.00138879, 0.00137701, 0.00135721,
       0.0013464 , 0.0013397 , 0.00133218, 0.00131365, 0.00129749,
       0.00128115, 0.00126958, 0.00126427, 0.00125678, 0.00124812,
       0.00122061, 0.00121139, 0.00120578, 0.00119165, 0.00118967,
       0.00117831, 0.00116613, 0.00116088, 0.00114835, 0.00113906,
       0.00112037, 0.00111411, 0.00111165, 0.00109484, 0.00109124,
       0.00108197, 0.00106722, 0.00106136, 0.00105476, 0.00104631,
       0.00103588, 0.00102664, 0.00102017, 0.00101175, 0.00100

In [29]:
sum(lsa_train.explained_variance_ratio_)

0.15602311771078764

In [30]:
import sys
sys.getsizeof(train_doc_topic)/1000000
# sys.getsizeof(lsa_train)


127.656912

In [32]:
topic_word = pd.DataFrame(lsa_train.components_.round(3),
             index = [f'component_{i}' for i in range (n)],
             columns = word_vectorizer.get_feature_names())
topic_word

,0,1,1500,a,aa,aaa,aachen,aagadu,aah,aaliyah,...,zvonimir,zwarte,zx,zydeco,zyklon,zyxoas,zzuuzz,zzz,zzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz
component_0,-0.0,-0.0,0.0,0.181,0.000,0.000,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0
component_1,0.0,0.0,-0.0,-0.084,-0.000,0.000,-0.0,0.0,-0.000,-0.0,...,-0.0,-0.0,0.0,-0.0,-0.0,0.000,0.001,0.0,-0.0,-0.0
component_2,0.0,-0.0,0.0,0.032,0.000,-0.000,-0.0,0.0,-0.000,-0.0,...,0.0,0.0,0.0,0.0,-0.0,-0.000,0.000,0.0,-0.0,-0.0
component_3,-0.0,-0.0,0.0,-0.007,0.000,-0.000,0.0,-0.0,-0.000,-0.0,...,0.0,0.0,-0.0,0.0,-0.0,-0.000,0.000,0.0,0.0,-0.0
component_4,-0.0,0.0,-0.0,0.010,-0.000,0.000,-0.0,-0.0,0.000,0.0,...,-0.0,0.0,-0.0,-0.0,0.0,-0.000,-0.001,0.0,0.0,0.0
component_5,-0.0,0.0,0.0,0.095,0.000,-0.000,0.0,-0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.000,-0.000,-0.0,0.0,-0.0
component_6,0.0,-0.0,-0.0,0.006,0.000,-0.000,-0.0,-0.0,-0.000,-0.0,...,0.0,0.0,0.0,0.0,-0.0,-0.000,-0.000,0.0,-0.0,0.0
component_7,-0.0,-0.0,-0.0,0.038,0.000,0.000,0.0,-0.0,-0.000,0.0,...,0.0,0.0,-0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0
component_8,-0.0,0.0,-0.0,0.018,0.000,0.000,0.0,0.0,0.000,0.0,...,0.0,0.0,-0.0,-0.0,0.0,-0.000,0.001,-0.0,-0.0,-0.0
component_9,0.0,-0.0,0.0,0.037,0.000,0.000,-0.0,-0.0,-0.000,0.0,...,0.0,0.0,-0.0,-0.0,0.0,0.000,-0.000,0.0,0.0,-0.0


In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
display_topics(lsa_train, vectorizer.get_feature_names(), 5)

In [ ]:
Vt = pd.DataFrame(train_doc_topic.round(5),
             index = train,
             columns = [f'component_{i}' for i in range (n)])

In [ ]:
Vt[Vt['component_0'] > 2].head(3)

In [ ]:
Vt[Vt['component_2'] > 2].head(3)

In [ ]:
Vt[Vt['component_3'] > 0.8].head(3)

In [ ]:
Vt[Vt['component_4'] > 0.8].head(3)

In [ ]:
Vt[Vt['component_5'] > 2].head(3)

In [ ]:
Vt[Vt['component_6'] > 2].head(3)

# Second attempt at a useful model

In [ ]:
import os
import gensim

# Path to where the word2vec file lives
google_vec_file = '/Users/randy/Documents/GitHub/data_files/GoogleNews-vectors-negative300.bin.gz'

In [ ]:
# Load it!  This might take a few minutes...
model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

In [ ]:
# We can access individual word vectors using a dictionary-like syntax
model['Metis']

In [ ]:
# Some cool results!
model.most_similar('gal' ,topn=8)

In [ ]:
model.most_similar('dude' ,topn=8)

In [ ]:
# Here's an analogy task!
# Positive words contribute positively towards the similarity, negative words negatively
model.most_similar(positive=['woman', 'king'], negative=['man']) #man is to king as woman is to...

# Third attempt at useful models

In [ ]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from scipy.sparse import hstack
from scipy.special import logit, expit

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
import string

import gensim

import pickle

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
import string

%matplotlib inline
#set column widths t
pd.set_option('display.max_colwidth', -1)

# Import and clean data

In [ ]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

raw = pd.read_csv('/Users/randy/Documents/GitHub/Twitch_Chat_Harassment/toxic_comment_classification/jigsaw-toxic-comment-classification-challenge/cleaned_train.csv').fillna(' ')
test = pickle.load(open('/Users/randy/Documents/GitHub/Twitch_Chat_Harassment/chat_logs/callofduty_chat.p','rb'))

In [ ]:
#check the first few rows. Note that there are a bunch of NaN columns
raw.head(3)

In [ ]:
data = raw.drop(columns=['id'])

In [ ]:
data['comment_text'] = data['comment_text'].str.lower()

In [ ]:
#check the dimensions
data.shape

In [ ]:
#remove non-ascii characters
def remove_non_ascii(text):
    text  = "".join([char for char in text if char in string.printable])
    text = re.sub('[0-9]+', '', text)
    return text

data['non_ascii_tweet'] = data['comment_text'].apply(lambda x: remove_non_ascii(x))
data.head(2)

In [ ]:
#remove the punctuation
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

data['no_punct_tweet'] = data['non_ascii_tweet'].apply(lambda x: remove_punct(x))
data.head()

In [ ]:
#tokenize the data
def tokenization(text):
    text = re.split('\W+', text)
    return text

data['Tweet_tokenized'] = data['no_punct_tweet'].apply(lambda x: tokenization(x.lower()))
data.head()

In [ ]:
#remove STOPWORDS
stopword = nltk.corpus.stopwords.words('english')
#stopword.extend(['yr', 'year', 'woman', 'man', 'girl','boy','one', 'two', 'sixteen', 'yearold', 'fu', 'weeks', 'week',
#               'treatment', 'associated', 'patients', 'may','day', 'case','old'])


In [ ]:
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
data['Tweet_nonstop'] = data['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))
data.head(5)

In [ ]:
#Stemming and Lammitization
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text if len(text) <30]
    return text

data['Tweet_stemmed'] = data['Tweet_nonstop'].apply(lambda x: stemming(x))
data.head()

In [ ]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text if len(word) <30]
    return text

data['Tweet_lemmatized'] = data['Tweet_nonstop'].apply(lambda x: lemmatizer(x))


In [ ]:
data.head(20)

In [ ]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

# Step 2: Vectorize (create countVectorizer and TF-IDF Vectorizer)

# Starting with countVectorizer:

In [ ]:
countVectorizer = CountVectorizer(analyzer=clean_text, ngram_range=(3,3), stop_words='english') 
X = countVectorizer.fit_transform(data['comment_text'])
# print('{} Number of tweets has {} words'.format(X.shape[0], X.shape[1]))
print(f"Dimensions of Document-term matrix: {X.toarray().shape}")
# print(X.get_feature_names())

In [ ]:
count_vect_df = pd.DataFrame(X.toarray(), columns=countVectorizer.get_feature_names())
count_vect_df.head()

# TF-IDF Vectorizer (sklearn method):

In [ ]:
# For sklearn, it's VERY similar to how we did CountVectorizer
vectorizer = TfidfVectorizer(analyzer=clean_text, ngram_range=(1,1), stop_words='english')
X_tfidf = vectorizer.fit_transform(data['comment_text'])

In [ ]:
# Tokenize the documents
# Convert a document into a list of lowercase tokens.
tokenized_docs = [gensim.utils.simple_preprocess(d) for d in data['comment_text']]

# Create a Gensim Dictionary.  This creates an id to word mapping for everything in our vocbulary
# It is NOT the same as the dictionary object in the Python standard library
# index is key, value is word
mydict = gensim.corpora.Dictionary()

# Create a Gensim Corpus object.  This creates a list of tuples for each document.
# The first element of the tuple is the word id, the second is the number of counts
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_docs]

In [ ]:
# This creates the doc-term matrix as a numpy array.
# Typically these matrices are HUGE so, it's usuall not a great idea to create the full dense doc-term matrix.
# We do it here to illustrate that you can get the same info as we obtained in scikit-learn!
doc_term_matrix = gensim.matutils.corpus2dense(mycorpus, num_terms=len(mydict))

In [ ]:
doc_term_matrix

In [ ]:
# Creating a tf-idf model is very simple!
tfidf = gensim.models.TfidfModel(mycorpus)
tfidf_matrix = gensim.matutils.corpus2dense(tfidf[mycorpus], num_terms=len(mydict))

In [ ]:
tfidf_matrix

# Train Models

# ExtraTreeClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

losses = []
predictions = {'id': test['id']}
models = {}
for class_name in class_names:
    train_target = train[class_name]
    classifier = ExtraTreesClassifier(n_estimators=30)

    cv_loss = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    losses.append(cv_loss)
    print('CV score for class {} is {}'.format(class_name, cv_loss))
    
    classifier.fit(train_features, train_target)
    models[class_name] = classifier
    predictions[class_name] = classifier.predict_proba(test_features)[:, 1]

In [ ]:
test_features.toarray()

In [ ]:
check = [['message','id','toxic','severe_toxic','obscene','threat','insult','identity_hate']]

for item in zip(*predictions.values()):
    check.append([list_sentences_test[item[0]],item[0],item[1],item[2],item[3],item[4],item[5],item[6]])

In [ ]:
x = pd.DataFrame(check)

In [ ]:
new_header = x.iloc[0] #grab the first row for the header
x = x[1:] #take the data less the header row
x.columns = new_header #set the header row as the df header

In [ ]:
x.shape

In [ ]:
x.columns

In [ ]:
models

In [ ]:
predictions.keys()

In [ ]:
#pickle the models
# Save Model as a pickle Using joblib
import pickle
from sklearn.externals import joblib

# Save the model as a pickle in a file 
joblib.dump(models, 'twitch_models.p') 

# Load the model from the file 
pickled_models = joblib.load('twitch_models.p')  
  


In [ ]:
pickled_models['toxic'].fit(train_features, train_target)
predictions['toxic'] = pickled_models['toxic'].predict_proba(test_features)[:, 1]

In [ ]:
toxic = pickled_models['toxic']

In [ ]:
cv_loss = np.mean(cross_val_score(toxic, train_features, train_target, cv=3, scoring='roc_auc'))
print('CV score for toxic class is {}'.format(cv_loss))